# Refer to this website to change the Company you want to scrape

https://github.com/robertlandlord/glassdoor_scraper

In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium import *
import xlwt
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

option = webdriver.ChromeOptions()
option.add_argument("—incognito")
prefs = {'profile.managed_default_content_settings.images':2, 'disk-cache-size': 4096}
option.add_experimental_option("prefs", prefs)

url_link = "https://www.glassdoor.com/Reviews/Google-Reviews-E9079_P1.htm"
company_info = xlwt.Workbook(encoding="utf-8")
name = input("What would you like to name the file?") + ".csv"

def scrape(base_url, workbook, name):
    worksheet = workbook.add_sheet("sheet1", cell_overwrite_ok=True)
    worksheet.write(0, 1, "Title")
    worksheet.write(0, 2, "Date Written")
    worksheet.write(0, 3, "Rating")
    worksheet.write(0, 4, "Current/Former")
    worksheet.write(0, 5, "Job Title")
    worksheet.write(0, 6, "Location")
    worksheet.write(0, 7, "Recommendation?")
    worksheet.write(0, 8, "Outlook")
    worksheet.write(0, 9, "Main Text")
    worksheet.write(0, 10, "Pros")
    worksheet.write(0, 11, "Cons")
    worksheet.write(0, 12, "Advice to management")
    # setting first url
    url = base_url
    chrome_path = r'C:/Users/Owner/Desktop/chromedriver.exe'
    driver = webdriver.Chrome(chrome_path, chrome_options=option)
    # logging in
    driver.get("https://www.glassdoor.com/profile/login_input.htm?userOriginHook=HEADER_SIGNIN_LINK")
    # entering username/password
    username = driver.find_element_by_name("username")
    username.send_keys("glassdoorScraper@gmail.com")
    password = driver.find_element_by_name("password")
    password.send_keys("glassdoor")
    driver.find_element_by_xpath("//*[@class='gd-btn gd-btn-1 fill']").click()
    
    # after login
    try:
        # go to first link
        driver.get(url)
        url_arr = url.split("_")
        basic = url_arr[0]+"_P"
        page_arr = list(url_arr[1])
        page = ""
        for char in page_arr:
            if char.isdigit():
                page += char
        # this is the current page we are on
        page = int(page)
        counter = 0
        while True:
            # search the company
            # ~~~~~~~~~~~~~~~~~~~TITLE OF REVIEW~~~~~~~~~~~~~~~~~~~~~~
            titles = driver.find_elements_by_class_name("reviewLink")
            titlearr = []
            for title in titles:
                counter += 1
                titlearr.append(title.text)

            # ~~~~~~~~~~~~~~~~~~~~TIMESTAMPS~~~~~~~~~~~~~~~~~~~~~~~
            timestamps = driver.find_elements_by_class_name("floatLt")
            datelist = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            true_timestamps = []
            for datetime in timestamps:
                if datetime.text[:3] in datelist:
                    true_timestamps.append(datetime.text)



            # ~~~~~~~~~~~~~~~~~~~RATINGS~~~~~~~~~~~~~~~~~~~~~~~~~
            ratings = driver.find_elements_by_class_name("value-title")
            ratingarr = []
            for rating in ratings:  # NEED TO REMOVE THE FIRST ONE BECAUSE ITS GETTING OVERALL RATING
                ratingarr.append(rating.get_attribute('title'))
            if int(page) == 1:
                del ratingarr[0]

            # ~~~~~~~~~~~~~~~~~~~~AUTHOR INFO~~~~~~~~~~~~~~~~~~~~~~~~
            total_author_info = driver.find_elements_by_xpath('//*[@class="authorInfo tbl hideHH"]')
            cfstatusarr = []
            posarr = []
            locarr = []
            for author_info in total_author_info:
                infolist = author_info.text.split(' - ')
                cfstatus = infolist[0]
                position = ""
                location = ""
                if " in " in infolist[1]:
                    pos_split = infolist[1].split(' in ')
                    position = pos_split[0]
                    location = pos_split[1]
                else:
                    position = infolist[1]
                cfstatusarr.append(cfstatus)
                posarr.append(position)
                locarr.append(location)

            # ~~~~~~~~~~~~~~~~~~~~~~~~~FULL REVIEW~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            triples = driver.find_elements_by_xpath('//*[@class="cell reviewBodyCell"]')
            recarr = []
            outlookarr = []
            othertextarr = []
            prosarr = []
            consarr = []
            advarr = []
            for triple in triples:
                recommends = ""
                outlook = ""
                #ceo = None
                othertext = ""
                pros = ""
                cons = ""
                advice = ""
                textlist = triple.text.splitlines()
                for index in range(0, len(textlist) - 1):
                    if "Recommend" in textlist[index]:
                        recommends = textlist[index]
                    elif "Outlook" in textlist[index]:
                        outlook = textlist[index]
                    #elif "CEO" in textlist[index]:
                    #    ceo = textlist[index]
                    elif "Pros" == textlist[index]:
                        pros = textlist[index + 1]
                        textlist[index + 1] = "got"
                    elif "Cons" == textlist[index]:
                        cons = textlist[index + 1]
                        textlist[index + 1] = "got"
                    elif "Advice to Management" in textlist[index]:
                        advice = textlist[index + 1]
                        textlist[index + 1] = "got"
                    elif "got" == textlist[index]:
                        continue
                    else:
                        othertext = textlist[index]
                recarr.append(recommends)
                outlookarr.append(outlook)
                othertextarr.append(othertext)
                prosarr.append(pros)
                consarr.append(cons)
                advarr.append(advice)


            for index in range(0, len(titlearr)):
                print("INDEX IS: ", index)
                print("ROW: ", int(page)*10+index)
                worksheet.write(int(page)*10+index, 1, titlearr[index])
                worksheet.write(int(page)*10+index, 2, true_timestamps[index])
                worksheet.write(int(page)*10+index, 3, ratingarr[index])
                worksheet.write(int(page)*10+index, 4, cfstatusarr[index])
                worksheet.write(int(page)*10+index, 5, posarr[index])
                worksheet.write(int(page)*10+index, 6, locarr[index])
                worksheet.write(int(page)*10+index, 7, recarr[index])
                worksheet.write(int(page)*10+index, 8, outlookarr[index])
                worksheet.write(int(page)*10+index, 9, othertextarr[index])
                worksheet.write(int(page)*10+index, 10, prosarr[index])
                worksheet.write(int(page)*10+index, 11, consarr[index])
                worksheet.write(int(page)*10+index, 12, advarr[index])
            try:
                found = driver.find_element_by_css_selector("#FooterPageNav > div > ul > li.page.current.last > span")
                print(found.text)
                workbook.save(name)
                break

            except NoSuchElementException:
                page += 1
                driver.get(basic+str(page)+".htm")
                print("clicked page", str(page))

    except NoSuchElementException:
        workbook.save(name)
        print("finished")


scrape(url_link, company_info, name)

What would you like to name the file?googletest1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: use options instead of chrome_options


INDEX IS:  0
ROW:  10
INDEX IS:  1
ROW:  11
INDEX IS:  2
ROW:  12
INDEX IS:  3
ROW:  13
INDEX IS:  4
ROW:  14
INDEX IS:  5
ROW:  15
INDEX IS:  6
ROW:  16
INDEX IS:  7
ROW:  17
INDEX IS:  8
ROW:  18
INDEX IS:  9
ROW:  19
clicked page 2
INDEX IS:  0
ROW:  20
INDEX IS:  1
ROW:  21
INDEX IS:  2
ROW:  22
INDEX IS:  3
ROW:  23
INDEX IS:  4
ROW:  24
INDEX IS:  5
ROW:  25
INDEX IS:  6
ROW:  26
INDEX IS:  7
ROW:  27
INDEX IS:  8
ROW:  28
INDEX IS:  9
ROW:  29
clicked page 3
INDEX IS:  0
ROW:  30
INDEX IS:  1
ROW:  31
INDEX IS:  2
ROW:  32
INDEX IS:  3
ROW:  33
INDEX IS:  4
ROW:  34
INDEX IS:  5
ROW:  35
INDEX IS:  6
ROW:  36
INDEX IS:  7
ROW:  37
INDEX IS:  8
ROW:  38
INDEX IS:  9
ROW:  39
clicked page 4
INDEX IS:  0
ROW:  40
INDEX IS:  1
ROW:  41
INDEX IS:  2
ROW:  42
INDEX IS:  3
ROW:  43
INDEX IS:  4
ROW:  44
INDEX IS:  5
ROW:  45
INDEX IS:  6
ROW:  46
INDEX IS:  7
ROW:  47
INDEX IS:  8
ROW:  48
INDEX IS:  9
ROW:  49
clicked page 5
INDEX IS:  0
ROW:  50
INDEX IS:  1
ROW:  51
INDEX IS:  2
ROW

clicked page 35
INDEX IS:  0
ROW:  350
INDEX IS:  1
ROW:  351
INDEX IS:  2
ROW:  352
INDEX IS:  3
ROW:  353
INDEX IS:  4
ROW:  354
INDEX IS:  5
ROW:  355
INDEX IS:  6
ROW:  356
INDEX IS:  7
ROW:  357
INDEX IS:  8
ROW:  358
INDEX IS:  9
ROW:  359
clicked page 36
INDEX IS:  0
ROW:  360
INDEX IS:  1
ROW:  361
INDEX IS:  2
ROW:  362
INDEX IS:  3
ROW:  363
INDEX IS:  4
ROW:  364
INDEX IS:  5
ROW:  365
INDEX IS:  6
ROW:  366
INDEX IS:  7
ROW:  367
INDEX IS:  8
ROW:  368
INDEX IS:  9
ROW:  369
clicked page 37
INDEX IS:  0
ROW:  370
INDEX IS:  1
ROW:  371
INDEX IS:  2
ROW:  372
INDEX IS:  3
ROW:  373
INDEX IS:  4
ROW:  374
INDEX IS:  5
ROW:  375
INDEX IS:  6
ROW:  376
INDEX IS:  7
ROW:  377
INDEX IS:  8
ROW:  378
INDEX IS:  9
ROW:  379
clicked page 38
INDEX IS:  0
ROW:  380
INDEX IS:  1
ROW:  381
INDEX IS:  2
ROW:  382
INDEX IS:  3
ROW:  383
INDEX IS:  4
ROW:  384
INDEX IS:  5
ROW:  385
INDEX IS:  6
ROW:  386
INDEX IS:  7
ROW:  387
INDEX IS:  8
ROW:  388
INDEX IS:  9
ROW:  389
clicked page 39


clicked page 69
INDEX IS:  0
ROW:  690
INDEX IS:  1
ROW:  691
INDEX IS:  2
ROW:  692
INDEX IS:  3
ROW:  693
INDEX IS:  4
ROW:  694
INDEX IS:  5
ROW:  695
INDEX IS:  6
ROW:  696
INDEX IS:  7
ROW:  697
INDEX IS:  8
ROW:  698
INDEX IS:  9
ROW:  699
clicked page 70
INDEX IS:  0
ROW:  700
INDEX IS:  1
ROW:  701
INDEX IS:  2
ROW:  702
INDEX IS:  3
ROW:  703
INDEX IS:  4
ROW:  704
INDEX IS:  5
ROW:  705
INDEX IS:  6
ROW:  706
INDEX IS:  7
ROW:  707
INDEX IS:  8
ROW:  708
INDEX IS:  9
ROW:  709
clicked page 71
INDEX IS:  0
ROW:  710
INDEX IS:  1
ROW:  711
INDEX IS:  2
ROW:  712
INDEX IS:  3
ROW:  713
INDEX IS:  4
ROW:  714
INDEX IS:  5
ROW:  715
INDEX IS:  6
ROW:  716
INDEX IS:  7
ROW:  717
INDEX IS:  8
ROW:  718
INDEX IS:  9
ROW:  719
clicked page 72
INDEX IS:  0
ROW:  720
INDEX IS:  1
ROW:  721
INDEX IS:  2
ROW:  722
INDEX IS:  3
ROW:  723
INDEX IS:  4
ROW:  724
INDEX IS:  5
ROW:  725
INDEX IS:  6
ROW:  726
INDEX IS:  7
ROW:  727
INDEX IS:  8
ROW:  728
INDEX IS:  9
ROW:  729
clicked page 73


...

clicked page 615
INDEX IS:  0
ROW:  6150
INDEX IS:  1
ROW:  6151
INDEX IS:  2
ROW:  6152
INDEX IS:  3
ROW:  6153
INDEX IS:  4
ROW:  6154
INDEX IS:  5
ROW:  6155
INDEX IS:  6
ROW:  6156
INDEX IS:  7
ROW:  6157
INDEX IS:  8
ROW:  6158
INDEX IS:  9
ROW:  6159
clicked page 616
INDEX IS:  0
ROW:  6160
INDEX IS:  1
ROW:  6161
INDEX IS:  2
ROW:  6162
INDEX IS:  3
ROW:  6163
INDEX IS:  4
ROW:  6164
INDEX IS:  5
ROW:  6165
INDEX IS:  6
ROW:  6166
INDEX IS:  7
ROW:  6167
INDEX IS:  8
ROW:  6168
INDEX IS:  9
ROW:  6169
clicked page 617
INDEX IS:  0
ROW:  6170
INDEX IS:  1
ROW:  6171
INDEX IS:  2
ROW:  6172
INDEX IS:  3
ROW:  6173
INDEX IS:  4
ROW:  6174
INDEX IS:  5
ROW:  6175
INDEX IS:  6
ROW:  6176
INDEX IS:  7
ROW:  6177
INDEX IS:  8
ROW:  6178
INDEX IS:  9
ROW:  6179
clicked page 618
INDEX IS:  0
ROW:  6180
INDEX IS:  1
ROW:  6181
INDEX IS:  2
ROW:  6182
INDEX IS:  3
ROW:  6183
INDEX IS:  4
ROW:  6184
INDEX IS:  5
ROW:  6185
INDEX IS:  6
ROW:  6186
INDEX IS:  7
ROW:  6187
INDEX IS:  8
ROW:  6

clicked page 647
INDEX IS:  0
ROW:  6470
INDEX IS:  1
ROW:  6471
INDEX IS:  2
ROW:  6472
INDEX IS:  3
ROW:  6473
INDEX IS:  4
ROW:  6474
INDEX IS:  5
ROW:  6475
INDEX IS:  6
ROW:  6476
INDEX IS:  7
ROW:  6477
INDEX IS:  8
ROW:  6478
INDEX IS:  9
ROW:  6479
clicked page 648
INDEX IS:  0
ROW:  6480
INDEX IS:  1
ROW:  6481
INDEX IS:  2
ROW:  6482
INDEX IS:  3
ROW:  6483
INDEX IS:  4
ROW:  6484
INDEX IS:  5
ROW:  6485
INDEX IS:  6
ROW:  6486
INDEX IS:  7
ROW:  6487
INDEX IS:  8
ROW:  6488
INDEX IS:  9
ROW:  6489
clicked page 649
INDEX IS:  0
ROW:  6490
INDEX IS:  1
ROW:  6491
INDEX IS:  2
ROW:  6492
INDEX IS:  3
ROW:  6493
INDEX IS:  4
ROW:  6494
INDEX IS:  5
ROW:  6495
INDEX IS:  6
ROW:  6496
INDEX IS:  7
ROW:  6497
INDEX IS:  8
ROW:  6498
INDEX IS:  9
ROW:  6499
clicked page 650
INDEX IS:  0
ROW:  6500
INDEX IS:  1
ROW:  6501
INDEX IS:  2
ROW:  6502
INDEX IS:  3
ROW:  6503
INDEX IS:  4
ROW:  6504
INDEX IS:  5
ROW:  6505
INDEX IS:  6
ROW:  6506
INDEX IS:  7
ROW:  6507
INDEX IS:  8
ROW:  6

clicked page 679
INDEX IS:  0
ROW:  6790
INDEX IS:  1
ROW:  6791
INDEX IS:  2
ROW:  6792
INDEX IS:  3
ROW:  6793
INDEX IS:  4
ROW:  6794
INDEX IS:  5
ROW:  6795
INDEX IS:  6
ROW:  6796
INDEX IS:  7
ROW:  6797
INDEX IS:  8
ROW:  6798
INDEX IS:  9
ROW:  6799
clicked page 680
INDEX IS:  0
ROW:  6800
INDEX IS:  1
ROW:  6801
INDEX IS:  2
ROW:  6802
INDEX IS:  3
ROW:  6803
INDEX IS:  4
ROW:  6804
INDEX IS:  5
ROW:  6805
INDEX IS:  6
ROW:  6806
INDEX IS:  7
ROW:  6807
INDEX IS:  8
ROW:  6808
INDEX IS:  9
ROW:  6809
clicked page 681
INDEX IS:  0
ROW:  6810
INDEX IS:  1
ROW:  6811
INDEX IS:  2
ROW:  6812
INDEX IS:  3
ROW:  6813
INDEX IS:  4
ROW:  6814
INDEX IS:  5
ROW:  6815
INDEX IS:  6
ROW:  6816
INDEX IS:  7
ROW:  6817
INDEX IS:  8
ROW:  6818
INDEX IS:  9
ROW:  6819
clicked page 682
INDEX IS:  0
ROW:  6820
INDEX IS:  1
ROW:  6821
INDEX IS:  2
ROW:  6822
INDEX IS:  3
ROW:  6823
INDEX IS:  4
ROW:  6824
INDEX IS:  5
ROW:  6825
INDEX IS:  6
ROW:  6826
INDEX IS:  7
ROW:  6827
INDEX IS:  8
ROW:  6

clicked page 711
INDEX IS:  0
ROW:  7110
INDEX IS:  1
ROW:  7111
INDEX IS:  2
ROW:  7112
INDEX IS:  3
ROW:  7113
INDEX IS:  4
ROW:  7114
INDEX IS:  5
ROW:  7115
INDEX IS:  6
ROW:  7116
INDEX IS:  7
ROW:  7117
INDEX IS:  8
ROW:  7118
INDEX IS:  9
ROW:  7119
clicked page 712
INDEX IS:  0
ROW:  7120
INDEX IS:  1
ROW:  7121
INDEX IS:  2
ROW:  7122
INDEX IS:  3
ROW:  7123
INDEX IS:  4
ROW:  7124
INDEX IS:  5
ROW:  7125
INDEX IS:  6
ROW:  7126
INDEX IS:  7
ROW:  7127
INDEX IS:  8
ROW:  7128
INDEX IS:  9
ROW:  7129
clicked page 713
INDEX IS:  0
ROW:  7130
INDEX IS:  1
ROW:  7131
INDEX IS:  2
ROW:  7132
INDEX IS:  3
ROW:  7133
INDEX IS:  4
ROW:  7134
INDEX IS:  5
ROW:  7135
INDEX IS:  6
ROW:  7136
INDEX IS:  7
ROW:  7137
INDEX IS:  8
ROW:  7138
INDEX IS:  9
ROW:  7139
clicked page 714
INDEX IS:  0
ROW:  7140
INDEX IS:  1
ROW:  7141
INDEX IS:  2
ROW:  7142
INDEX IS:  3
ROW:  7143
INDEX IS:  4
ROW:  7144
INDEX IS:  5
ROW:  7145
INDEX IS:  6
ROW:  7146
INDEX IS:  7
ROW:  7147
INDEX IS:  8
ROW:  7

clicked page 743
INDEX IS:  0
ROW:  7430
INDEX IS:  1
ROW:  7431
INDEX IS:  2
ROW:  7432
INDEX IS:  3
ROW:  7433
INDEX IS:  4
ROW:  7434
INDEX IS:  5
ROW:  7435
INDEX IS:  6
ROW:  7436
INDEX IS:  7
ROW:  7437
INDEX IS:  8
ROW:  7438
INDEX IS:  9
ROW:  7439
clicked page 744
INDEX IS:  0
ROW:  7440
INDEX IS:  1
ROW:  7441
INDEX IS:  2
ROW:  7442
INDEX IS:  3
ROW:  7443
INDEX IS:  4
ROW:  7444
INDEX IS:  5
ROW:  7445
INDEX IS:  6
ROW:  7446
INDEX IS:  7
ROW:  7447
INDEX IS:  8
ROW:  7448
INDEX IS:  9
ROW:  7449
clicked page 745
INDEX IS:  0
ROW:  7450
INDEX IS:  1
ROW:  7451
INDEX IS:  2
ROW:  7452
INDEX IS:  3
ROW:  7453
INDEX IS:  4
ROW:  7454
INDEX IS:  5
ROW:  7455
INDEX IS:  6
ROW:  7456
INDEX IS:  7
ROW:  7457
INDEX IS:  8
ROW:  7458
INDEX IS:  9
ROW:  7459
clicked page 746
INDEX IS:  0
ROW:  7460
INDEX IS:  1
ROW:  7461
INDEX IS:  2
ROW:  7462
INDEX IS:  3
ROW:  7463
INDEX IS:  4
ROW:  7464
INDEX IS:  5
ROW:  7465
INDEX IS:  6
ROW:  7466
INDEX IS:  7
ROW:  7467
INDEX IS:  8
ROW:  7